In [1]:
import sys
sys.path.insert(0, '..')
print(sys.path)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
print(sys.path)
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
import json

['..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/ubuntu/.ipython']
['..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/ubuntu/.ipython', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model']


In [2]:
clean_df = pd.read_csv("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference/test.csv")
clean_df.head()

,Unnamed: 0,test_caption,test_imgs
0,0,feiyangg/LP Paragraph Style Electric Guitar Ti...,O1CN01r2RqZk1lJ4Sk2ZYaN_!!2200643934797.jpg
1,1,Summer Lace White Shorts Pants Female Thin Ant...,O1CN01H0o2pi2HAtGC9rVWa_!!2206736519111.jpg
2,2,High-Quality Export 8MM with Adhesive Cork Cor...,O1CN01qeHW0Z1D3mRPfmNvM_!!161-0-lubanu.jpg
3,3,LED Ceiling Living Room Bedroom Corridor Corri...,O1CN01phQoOe25ljjN1lpLO_!!3082327567.jpg
4,4,Disposable Cotton Cotton Female Thin Water Fac...,O1CN01Dzpgpr2FxJsUzqAdE_!!2963188946.jpg


In [3]:
image_embed_path = "./image_embeds"
text_embed_path = "./text_embeds"
image_embed_files = os.listdir(image_embed_path)
text_embed_files = os.listdir(text_embed_path)

In [4]:
all_image_embed ={}
all_text_embed = {}
for i,t in tqdm(zip(image_embed_files,text_embed_files)):
    with open(f"{image_embed_path}/{i}","rb") as img_file:
        img_embed = pickle.load(img_file)
        all_image_embed[img_embed["model_name"]] = img_embed["embed"]
    with open(f"{text_embed_path}/{t}","rb") as txt_file:
        txt_embed = pickle.load(txt_file)
        all_text_embed[txt_embed["model_name"]] = txt_embed["embed"]


4it [00:00,  6.21it/s]


In [5]:
all_image_embed.keys()

dict_keys(['base_all_e_7', 'base_2nd_e_5', 'base_1st_e_5', 'large_all_e_9'])

In [6]:
all_text_embed.keys()

dict_keys(['base_1st_e_5', 'base_2nd_e_5', 'base_all_e_7', 'large_all_e_9'])

In [7]:
pair_to_test = "large_all_e_9"

In [8]:
# embed_to_ensamble = ["base_1st_e_5","base_2nd_e_5"]

embed_to_ensamble = list(all_text_embed.keys())

In [9]:
# all_text_embed["base_all_e_7"] = all_text_embed["base_all_e_7"][0:20000]

In [10]:
def get_ensamble(all_image_embed,all_text_embed,embeds_to_ensamble,ensamble_strategy):
    ensamsables = {}
    img_embed_list = []
    text_embed_list = []
    for model_name in embeds_to_ensamble:
        img_embed_list.append(all_image_embed[model_name])
        text_embed_list.append(all_text_embed[model_name])
    if len(embeds_to_ensamble) > 1:
        if ensamble_strategy =="concat":
            ensamsables["image"] = torch.concat(img_embed_list,dim=1)
            ensamsables["text"] = torch.concat(text_embed_list,dim=1)
        elif ensamble_strategy =="avg":
            ensamsables["image"] = torch.mean(torch.stack(img_embed_list))
            ensamsables["text"] = torch.mean(torch.concat(text_embed_list))
        return ensamsables
    else:
            ensamsables["image"] = img_embed_list
            ensamsables["text"] = text_embed_list
            return ensamsables


        


In [11]:
ensamsables = get_ensamble(all_image_embed,all_text_embed,embed_to_ensamble,"concat")

In [12]:
test = ensamsables["text"] @ ensamsables["image"].T

In [13]:
torch.topk(test,10)

torch.return_types.topk(
values=tensor([[1.7092, 1.7044, 1.7009,  ..., 1.6379, 1.6238, 1.6073],
        [1.7183, 1.7036, 1.6723,  ..., 1.5285, 1.5083, 1.4777],
        [1.5418, 1.4432, 1.3890,  ..., 1.3541, 1.3369, 1.3197],
        ...,
        [1.6374, 1.5221, 1.5073,  ..., 1.4549, 1.4521, 1.4498],
        [1.7417, 1.6889, 1.6012,  ..., 1.4738, 1.4729, 1.4722],
        [1.5887, 1.5627, 1.5051,  ..., 1.3952, 1.3920, 1.3840]]),
indices=tensor([[27694, 40609, 47178,  ..., 46458, 42193, 34667],
        [23383, 44309,  5591,  ..., 22510,   582, 28545],
        [31777, 16947, 46953,  ..., 10273,  4198, 45215],
        ...,
        [24440, 42947, 43062,  ...,  8586,  8216, 26535],
        [ 4278, 18451, 39630,  ..., 20235, 27915, 11201],
        [43082, 35529,  6883,  ..., 14069, 42845, 27301]]))

In [14]:

def get_preds(ensamsables,score=False,df=None):
    image_embed = ensamsables["image"]
    text_embed = ensamsables["text"]
    faiss_index = faiss.IndexFlatIP(image_embed.shape[1])
    faiss_index.add(image_embed.numpy().astype(np.float32))
    top5_k_e,top5_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),5)
    top10_k_e,top10_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),10)
    if score:
        top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
        top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])
        top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
        print(f"top_5 accuracy:{top5_acc}")
        top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
        avg_acc = (top5_acc+top10_acc)/2
        print(f"top_5 accuracy:{top10_acc}")
        print(f"avg accuracy:{avg_acc}")
        return top5_acc,top10_acc,avg_acc
    else:
        return top5_k_y_pred,top10_k_y_pred,top5_k_e,top10_k_e

In [15]:
top5_pred_idx,top10_pred_idx,top5_k_e,top10_k_e = get_preds(ensamsables,score=False,df=clean_df)

In [16]:
top10_k_e[0]

array([1.709192 , 1.7043643, 1.7009404, 1.6768483, 1.6623586, 1.6431701,
       1.6427323, 1.637881 , 1.6237569, 1.6073489], dtype=float32)

In [17]:
def remap_pred_to_name(top5_pred_idx,top10_pred_idx,image_array):
    top5_pred = [list(image_array[p]) for p in top5_pred_idx]
    top10_pred =  [list(image_array[p]) for p in top10_pred_idx]
    return top5_pred,top10_pred

In [18]:
top5_pred,top10_pred =remap_pred_to_name(top5_pred_idx,top10_pred_idx,clean_df["test_imgs"].values)

In [20]:
def results_to_json(top5_pred,top10_pred,caption_array,result_dir,result_file_name):
    results = []
    for c,t5,t10 in tqdm(zip(caption_array,top5_pred,top10_pred)):
        result_dict = {}
        result_dict["caption"] = c
        result_dict["top5"] = t5
        result_dict["top10"] = t10
        results.append(result_dict)
    print("checking results format")
    print(results[0])
    with open(f"{result_dir}/{result_file_name}.json","w") as results_file:
        json.dump(results,results_file)
    with open(f"{result_dir}/{result_file_name}.json","r") as results_file:
        check_results = json.load(results_file)
    print("checking results format of saved file")
    print(check_results[-1])
    return check_results
    
        



In [21]:
saved_result = results_to_json(top5_pred,top10_pred,clean_df["test_caption"].values[0:20000],"./submissions","test_ensamble_all_3_6_22")

20000it [00:00, 909373.63it/s]


checking results format
{'caption': 'feiyangg/LP Paragraph Style Electric Guitar Tiger Veneer Factory Direct Color Can Be Customized', 'top5': ['TB25xAzdCGI.eBjSspcXXcVjFXa_!!754885367.jpg', 'O1CN01Dj2czX1oWlFVAZ05k_!!2207737885233.jpg', 'O1CN011pW88rTaqfnNIi1_!!754885367.jpg', 'O1CN01S8ewME1m1UwiK89Pr_!!1973994894.jpg', 'O1CN01p912eO1lKuB1TiTl6_!!2207916444801.png'], 'top10': ['TB25xAzdCGI.eBjSspcXXcVjFXa_!!754885367.jpg', 'O1CN01Dj2czX1oWlFVAZ05k_!!2207737885233.jpg', 'O1CN011pW88rTaqfnNIi1_!!754885367.jpg', 'O1CN01S8ewME1m1UwiK89Pr_!!1973994894.jpg', 'O1CN01p912eO1lKuB1TiTl6_!!2207916444801.png', 'O1CN016zxVN01qPYIlq8the_!!0-item_pic.jpg', 'O1CN01ha825u1ctPVv8XwDP_!!2211470413658-0-picasso.jpg', 'O1CN01h0ZgaM1qPYIr3q9Cf_!!0-item_pic.jpg', 'O1CN01XJH6iU1lKuBgk0ndT_!!2207916444801.jpg', 'TB2lylSaQ.OyuJjSszhXXbZbVXa_!!20295181.jpg']}
checking results format of saved file
{'caption': 'mrs harmay foods Handmade Cantonese Barbecued Pork Bag 390g/Bag', 'top5': ['O1CN019zeHtc2BQ6VwvUSxE_!!2

In [22]:
len(saved_result)

20000

In [23]:
saved_result[:1]

[{'caption': 'feiyangg/LP Paragraph Style Electric Guitar Tiger Veneer Factory Direct Color Can Be Customized',
  'top5': ['TB25xAzdCGI.eBjSspcXXcVjFXa_!!754885367.jpg',
   'O1CN01Dj2czX1oWlFVAZ05k_!!2207737885233.jpg',
   'O1CN011pW88rTaqfnNIi1_!!754885367.jpg',
   'O1CN01S8ewME1m1UwiK89Pr_!!1973994894.jpg',
   'O1CN01p912eO1lKuB1TiTl6_!!2207916444801.png'],
  'top10': ['TB25xAzdCGI.eBjSspcXXcVjFXa_!!754885367.jpg',
   'O1CN01Dj2czX1oWlFVAZ05k_!!2207737885233.jpg',
   'O1CN011pW88rTaqfnNIi1_!!754885367.jpg',
   'O1CN01S8ewME1m1UwiK89Pr_!!1973994894.jpg',
   'O1CN01p912eO1lKuB1TiTl6_!!2207916444801.png',
   'O1CN016zxVN01qPYIlq8the_!!0-item_pic.jpg',
   'O1CN01ha825u1ctPVv8XwDP_!!2211470413658-0-picasso.jpg',
   'O1CN01h0ZgaM1qPYIr3q9Cf_!!0-item_pic.jpg',
   'O1CN01XJH6iU1lKuBgk0ndT_!!2207916444801.jpg',
   'TB2lylSaQ.OyuJjSszhXXbZbVXa_!!20295181.jpg']}]

In [24]:
top5_pred_idx

array([[27694, 40609, 47178, 28805, 35939],
       [23383, 44309,  5591, 19321, 16976],
       [31777, 16947, 46953, 34609, 14996],
       ...,
       [24440, 42947, 43062,  7107, 31868],
       [ 4278, 18451, 39630, 12274, 20602],
       [43082, 35529,  6883, 12174, 26153]])